In [87]:
from langgraph_supervisor import create_supervisor
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver 
from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_handoff_tool, create_swarm
from typing import List
import os
load_dotenv()

True

In [88]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
)

In [89]:
def question_answering(question: str):
    """General Question Answering Agent

    Handles non-scientific, general knowledge questions.
    Transfers science questions to science_agent and translation requests to translator_agent.

    Args:
        question (str): The user's question

    Returns:
        str: Response from the language model
    """
    response = llm.invoke(question)
    return response.content

def science_agent(question: str):
    """Science Specialist Agent

    Handles questions about physics, chemistry, biology, astronomy, and other scientific topics.
    Transfers non-science questions to question_answering_agent and translation requests to translator_agent.

    Args:
        question (str): The science-related question

    Returns:
        str: Scientific explanation or answer
    """
    response = llm.invoke(question)
    return response.content

def extract_language_and_text(request: str):
    """Extract target language and text from a translation request

    Parses the user's request to identify:
    1. The text to be translated
    2. The target language for translation

    Handles various formats:
    - "translate X to Y"
    - "X in Y"
    - "X into Y"

    Args:
        request (str): The user's translation request

    Returns:
        tuple: (text_to_translate, target_language)
    """
    request = request.lower()

    # Common language patterns
    to_patterns = [" to ", " in ", " into "]

    # Try to find language in the request
    target_language = None
    text = request

    for pattern in to_patterns:
        if pattern in request:
            parts = request.split(pattern)
            if len(parts) == 2:
                text = parts[0]
                # Take everything after the pattern as language
                target_language = parts[1].strip()
                break

    # Clean up text by removing translation-related prefixes
    text = text.replace("translate", "").replace("say", "").strip()
    return text.strip(), target_language

def translate_text(request: str):
    """Translation Handler

    Processes translation requests in various formats:
    1. Direct translation with language: "translate X to Y"
    2. Help requests: "help with translation"
    3. Text-only requests: Prompts for target language

    Args:
        request (str): The translation request or text to translate

    Returns:
        str: Either translation instructions or the translated text
    """
    # If it's a help request, provide format guidance
    if any(x in request.lower() for x in ["help with", "can you", "need translation", "translate something"]):
        return "I can help you translate. Please provide what you want to translate using this format:\n'translate [your text] to [language]' or '[text] in [language]'"

    # Try to extract language and text
    text, target_language = extract_language_and_text(request)

    # If language wasn't found in the request, ask for it
    if not target_language:
        target_language = input("Which language would you like this translated to? ")

    prompt = f"Translate the following text to {target_language}:\n{text}"
    response = llm.invoke(prompt)
    return response.content


In [90]:
physics_agent = create_react_agent(
    model=llm,
    tools=[
        create_handoff_tool(agent_name="chemistry_agent"),
        create_handoff_tool(agent_name="biology_agent"),
        create_handoff_tool(agent_name="astronomy_agent"),
        create_handoff_tool(agent_name="general_science_agent")
    ],
    name="physics_agent",
    prompt=(
        "You are a Physics expert specializing in mechanics, thermodynamics, quantum physics, "
        "relativity, electromagnetism, and all physics-related topics. "
        "If the question involves chemistry, transfer to chemistry_agent. "
        "If it involves biology, transfer to biology_agent. "
        "If it involves astronomy/space, transfer to astronomy_agent. "
        "If it's general science or interdisciplinary, transfer to general_science_agent. "
        "Provide detailed physics explanations with formulas and examples when appropriate."
    )
)

# 2. Chemistry Agent
chemistry_agent = create_react_agent(
    model=llm,
    tools=[
        create_handoff_tool(agent_name="physics_agent"),
        create_handoff_tool(agent_name="biology_agent"),
        create_handoff_tool(agent_name="astronomy_agent"),
        create_handoff_tool(agent_name="general_science_agent")
    ],
    name="chemistry_agent",
    prompt=(
        "You are a Chemistry expert specializing in organic chemistry, inorganic chemistry, "
        "physical chemistry, biochemistry, and all chemistry-related topics. "
        "If the question involves physics principles, transfer to physics_agent. "
        "If it involves biological systems, transfer to biology_agent. "
        "If it involves space/astronomy, transfer to astronomy_agent. "
        "If it's general science or interdisciplinary, transfer to general_science_agent. "
        "Provide detailed chemical explanations with reactions and molecular structures when appropriate."
    )
)

# 3. Biology Agent
biology_agent = create_react_agent(
    model=llm,
    tools=[
        create_handoff_tool(agent_name="physics_agent"),
        create_handoff_tool(agent_name="chemistry_agent"),
        create_handoff_tool(agent_name="astronomy_agent"),
        create_handoff_tool(agent_name="general_science_agent")
    ],
    name="biology_agent",
    prompt=(
        "You are a Biology expert specializing in molecular biology, genetics, ecology, "
        "evolution, physiology, and all biology-related topics. "
        "If the question involves physical principles, transfer to physics_agent. "
        "If it involves chemical processes, transfer to chemistry_agent. "
        "If it involves astrobiology or space, transfer to astronomy_agent. "
        "If it's general science or interdisciplinary, transfer to general_science_agent. "
        "Provide detailed biological explanations with examples from nature when appropriate."
    )
)

# 4. Astronomy Agent
astronomy_agent = create_react_agent(
    model=llm,
    tools=[
        create_handoff_tool(agent_name="physics_agent"),
        create_handoff_tool(agent_name="chemistry_agent"),
        create_handoff_tool(agent_name="biology_agent"),
        create_handoff_tool(agent_name="general_science_agent")
    ],
    name="astronomy_agent",
    prompt=(
        "You are an Astronomy expert specializing in astrophysics, cosmology, planetary science, "
        "stellar evolution, and all astronomy-related topics. "
        "If the question involves fundamental physics, transfer to physics_agent. "
        "If it involves chemical compositions, transfer to chemistry_agent. "
        "If it involves life in space (astrobiology), transfer to biology_agent. "
        "If it's general science or interdisciplinary, transfer to general_science_agent. "
        "Provide detailed astronomical explanations with cosmic examples and scales."
    )
)

# 5. General Science Agent (Coordinator)
general_science_agent = create_react_agent(
    model=llm,
    tools=[
        create_handoff_tool(agent_name="physics_agent"),
        create_handoff_tool(agent_name="chemistry_agent"),
        create_handoff_tool(agent_name="biology_agent"),
        create_handoff_tool(agent_name="astronomy_agent")
    ],
    name="general_science_agent",
    prompt=(
        "You are a General Science coordinator specializing in interdisciplinary science questions "
        "and scientific methodology. You handle:\n"
        "1. Questions that span multiple scientific disciplines\n"
        "2. General scientific concepts and methodology\n"
        "3. Science history and philosophy\n"
        "4. Questions where the specific field isn't clear\n\n"
        "Route specific questions to specialists:\n"
        "- Physics questions → physics_agent\n"
        "- Chemistry questions → chemistry_agent\n"
        "- Biology questions → biology_agent\n"
        "- Astronomy questions → astronomy_agent\n\n"
        "For interdisciplinary questions, provide a comprehensive answer or coordinate with specialists."
    )
)


In [91]:
science_swarm_checkpoint = InMemorySaver()
science_swarm = create_swarm(
    agents=[
        general_science_agent,
        physics_agent,
        chemistry_agent,
        biology_agent,
        astronomy_agent
    ],
    default_active_agent="general_science_agent",
)
science_swarm_app = science_swarm.compile(checkpointer=science_swarm_checkpoint)


In [92]:
image = science_swarm_app.get_graph().draw_mermaid_png()
with open("swarm.png", "wb") as f:
    f.write(image)

In [93]:
def invoke_science_swarm(question: str) -> str:
    """
    Invoke the science swarm to answer scientific questions.
    
    Args:
        question: The scientific question to answer
    
    Returns:
        The answer from the science swarm
    """
    config = {"configurable": {"thread_id": f"science_{hash(question)}"}}
    response = science_swarm_app.invoke(
        {"messages": [{"role": "user", "content": question}]},
        config # type: ignore
    )
    # Extract the final answer from the response
    final_message = response["messages"][-1]
    return final_message.content


In [94]:
question_answering_agent = create_react_agent(
    model=llm,
    tools=[question_answering, create_handoff_tool(agent_name="science_router_agent"), create_handoff_tool(agent_name="translator_agent")],
    name="question_answering_agent",
    prompt=(
        "You are a general question answering agent. For any questions related to science, physics, chemistry, "
        "biology, astronomy, or other scientific topics, you MUST use the handoff tool to transfer the question "
        "to the science_agent. For any translation requests, transfer to translator_agent. "
        "Only answer general knowledge questions yourself."
    )
)

science_router_agent = create_react_agent(
    model=llm,
    tools=[
        invoke_science_swarm,
        create_handoff_tool(agent_name="question_answering_agent"),
        create_handoff_tool(agent_name="translator_agent")
    ],
    name="science_router_agent",
    prompt=(
        "You are the science router agent. When you receive a science question, "
        "you delegate it to the specialized science swarm which consists of:\n"
        "- Physics Agent: mechanics, energy, forces, quantum physics\n"
        "- Chemistry Agent: reactions, molecules, compounds\n"
        "- Biology Agent: life, genetics, ecology\n"
        "- Astronomy Agent: space, stars, planets, cosmology\n"
        "- General Science Agent: interdisciplinary and methodology\n\n"
        "For NON-science questions, transfer to question_answering_agent.\n"
        "For translation requests, transfer to translator_agent.\n\n"
    )
)

translator_agent = create_react_agent(
    model=llm,
    tools=[translate_text, create_handoff_tool(agent_name="question_answering_agent"), create_handoff_tool(agent_name="science_router_agent")],
    prompt=(
        "You are a language translation agent. Your primary task is to help with translations:\n"
        "1. When user asks for help with translation, show them the format examples\n"
        "2. When user provides text with language (e.g., 'translate hello to spanish' or 'hello in spanish'), translate immediately\n"
        "3. When user provides only text, ask for the target language\n"
        "4. For questions about languages or translation in general, answer them directly\n"
        "5. For science questions, transfer to science_agent\n"
        "6. For other general questions, transfer to question_answering_agent\n"
        "Always aim for a natural single-step translation process."
    ),
    name="translator_agent"
)

In [95]:
main_checkpoint = InMemorySaver()
main_workflow = create_swarm(
    agents=[
        question_answering_agent,
        translator_agent,
        science_router_agent
    ],
    default_active_agent="question_answering_agent",
)
main_app = main_workflow.compile(checkpointer=main_checkpoint)

In [96]:
image = main_app.get_graph().draw_mermaid_png()
with open("swarm.png", "wb") as f:
    f.write(image)

In [97]:
config = {"configurable": {"thread_id": "1"}}

# Main interaction loop
while True:
    user_input = input("\nEnter your request (or 'exit' to quit): ")

    if user_input.lower() == 'exit':
        print("Goodbye!")
        break

    result = main_app.invoke(
        {"messages": [{
            "role": "user",
            "content": user_input
        }]},
        config, #type: ignore
    )

    # Display the response
    for m in result["messages"]:
        print(m.pretty_print())

================================ Human Message =================================

Explain quantum entanglement
None
================================== Ai Message ==================================
Name: question_answering_agent
Tool Calls:
  transfer_to_science_router_agent (call_nSloYoyOuavMWlpUyn3NzWJ6)
 Call ID: call_nSloYoyOuavMWlpUyn3NzWJ6
  Args:
None
================================= Tool Message =================================
Name: transfer_to_science_router_agent

Successfully transferred to science_router_agent
None
================================== Ai Message ==================================
Name: science_router_agent
Tool Calls:
  invoke_science_swarm (call_kiy9YDQfQFW7g54YsNpdp448)
 Call ID: call_kiy9YDQfQFW7g54YsNpdp448
  Args:
    question: Explain quantum entanglement.
None
================================= Tool Message =================================
Name: invoke_science_swarm

Quantum entanglement is a fascinating phenomenon in quantum mechanics where two or m